## 证照识别V2.0
1. 由于现在的证照的与原来的证照的图片有点不一样, 上次的的营业执照的图片有一个长方形的图片在外面,可以很好的识别出来这个,
2. 现在的营业执照上面也有一个长方形的外框,不过在这个外壳上面有一个国徽.
3. 如果和以前一样识别那种长方形的框,这个框会识别不出来.
4. 觉得采用新的识别方式,先识别出有字的地方,然后在识别出对应的字
5. 参考:https://blog.csdn.net/dcrmg/article/details/52095508
6. 步骤如下:
    1. 先识别二维码的信息
    2. 在识别具体的信息
        1. 原图像大小调整，提高运算效率
        2. 转化为灰度图
        3. 高斯平滑滤波
        4. 求得水平和垂直方向灰度图像的梯度差,使用Sobel算子
        5. 均值滤波，消除高频噪声
        6. 二值化
        7. 闭运算，填充条形码间隙
        8. 腐蚀，去除孤立的点
        9. 膨胀，填充条形码间空隙，根据核的大小，有可能需要2~3次膨胀操作
        10. 通过findContours找到条形码区域的矩形边界


In [47]:
import cv2
import numpy as np
import pyzbar.pyzbar as pz
import pytesseract
pytesseract.pytesseract.tesseract_cmd = "C:/Program Files (x86)/Tesseract-OCR/tesseract.exe"

In [110]:
source_img = cv2.imread(r"F:\PYcode\Coding\ic\license\source_img\4.jpg", cv2.IMREAD_COLOR)
img1 = 255 - source_img 

In [105]:
down_img = cv2.pyrDown(source_img)
re_down_img = cv2.pyrDown(down_img)

In [106]:
# 识别时候用的图片
img2 = cv2.cvtColor(re_down_img, cv2.COLOR_BGR2GRAY)
_, img3 = cv2.threshold(img2,125,255, cv2.THRESH_BINARY)
# img3 = cv2.morphologyEx(threshold,cv2.MORPH_CLOSE, (3,3))
# img3 = cv2.erode(img3, (5,5), iterations = 2) 
# img3 = cv2.dilate(imge, (3,3), iterations = 5)

In [109]:
if pz.decode(source_img):    
    for i in pz.decode(source_img):
        print(i.data.decode('utf-8'))
        print(i.rect)
        print(i.polygon)

统一社会信用代码：91510100MA63DEFJ00
企业注册号：510109001599590
名称：成都西辰易督科技有限公司
登记机关：成都市高新工商行政管理局
登记日期：2019-01-23
企业信用信息公示系统网址：http://www.cdgs.gov.cn  http://gsxt.cdcredit.gov.cn
Rect(left=928, top=3282, width=372, height=374)
[Point(x=928, y=3649), Point(x=1295, y=3656), Point(x=1300, y=3287), Point(x=935, y=3282)]


In [111]:
# 图片转灰色
img = cv2.cvtColor(img1, cv2.COLOR_RGB2GRAY)

In [112]:
# 高斯平滑滤波
img = cv2.GaussianBlur(img,(25,25), 0)
# bur = cv2.blur(img, (7,7))

In [113]:
# 求得水平和垂直方向灰度图像的梯度差,使用Sobel算子
#  sobelx = cv2.Sobel(frame,cv2.CV_64F,1,0,ksize=5)
imageX = cv2.Sobel(img, cv2.CV_64F,1,0,ksize=5)
imageY = cv2.Sobel(img, cv2.CV_64F,0,1,ksize=5)
# convertScaleAbs(imageX16S,imageSobelX,1,0);
imageSobelX = cv2.convertScaleAbs(imageX, 1.0)
imageSobelY = cv2.convertScaleAbs(imageY, 1.0)
imageSobelOut = imageSobelX + imageSobelY
# 相减不成功，相加成功

In [114]:
# 均值滤波，消除高频噪声 blur
bur = cv2.blur(imageSobelOut, (25,25))

In [115]:
# 二值化 threshold
retval,threshold = cv2.threshold(bur,125,255,cv2.THRESH_BINARY + cv2.THRESH_OTSU)

In [116]:
# 闭运算，
kernel1 = np.ones((5,5),np.uint8)  
imgc = cv2.morphologyEx(threshold,cv2.MORPH_CLOSE, kernel1)

In [117]:
# 腐蚀，去除孤立的点 erode
kernel2 = np.ones((15,15),np.uint8) 
imge = cv2.erode(imgc, kernel2, iterations = 2) 

In [118]:
# 膨胀，填充条形码间空隙，根据核的大小，有可能需要2~3次膨胀操作 dilate
imgd = cv2.dilate(imge, kernel1, iterations = 15)

In [119]:
# 通过findContours找到条形码区域的矩形边界
# contours, hierarchy = cv2.findContours(imgd, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)

a, contours, hierarchy = cv2.findContours(imgd,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_NONE) 

In [120]:
box_list = []
i = 1
for contour in contours:
    if cv2.contourArea(contour) < 100:
        continue
    else:
        rect = cv2.minAreaRect(contour)
        box = cv2.boxPoints(rect)
        box = np.int0(box)
        cv2.drawContours(img1, [box], 0, (0, 0, 255), 2)
        box_list.append(box)
        x, y, w, h = cv2.boundingRect(box)
        resout = pytesseract.image_to_string(img1[ y:(y+h), x:(x+w)], lang='chi_sim')
        if resout:
            cv2.imwrite("img_out\q%s.jpg"%(i), img1[ y:(y+h), x:(x+w)])
            print("img%s"%(i), resout)
        i += 1


img3 i 技 2 志 5 朔

E 吊 梁 伟
及 养 他 侬 法 应 当 公 定 的 信 息 应 在 信 息 吴 坂 房 20 个 工 作 月 内 公 示 ,
img4 水
img12 0 [ 命 国 丿 育 仪 明 国
img13 【
img17 2019 年 1 月 23 日
img19 卜
img22 A 吴 胡 育 的 月
img24 巫 人 水
img25 233

医
E
img27 DC 命 匹 俊 政 园 人
img30 木

司 ( 自 然 人 投 资 或 控 股 的 法 人 独 资 )
img34 [
img39 C 5 一 园 5
img41 欣 2 中 2 收
img51 E 吴 人 吴 伟 东 育 八 沥 吴 动 仁 技 一 一 连 浩

2

仪 椿
E ^
C
[ _
1

0 不 u 呆
E 河 0 宁 才 5 人 经相关部门批准后方可开展经蒜蒲动〕 s

1 吴 沥 [ 技 江 [ 口水 疫 诊

L

s 《

n 吊 吴 s 商 口 [ 吴 口 口 命 页 园 国 囡 吴 圆 园 唐 呆

T r - :
g

中 华 人 民 共 和 国 国 家 一 商 行 政 管 理 总 局 监 制


In [103]:
cv2.imwrite('save3.jpg',img3)

True

In [ ]:
img2gray = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)

imageVar = cv2.Laplacian(img2gray, cv2.CV_64F).var()
cv2.imshow('imageVar',imageVar)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [43]:
x, y, w, h = cv2.boundingRect(box_list[1])
x, y, w, h

(3111, 4574, 87, 34)

In [45]:
resout = pytesseract.image_to_string(img1[y:(y+h),x:(x+w)])

In [31]:
imgxx = img1[860:1250, 900:2700]

In [32]:
# equ = cv2.equalizeHist(imgg) # 只能传入灰度图
# res = np.hstack((imgg,equ))  # 图像列拼接（用于显示）
cv2.imshow('imgxx',imgxx)
cv2.waitKey(0)
cv2.destroyAllWindows()

### 结束
------
***


In [ ]:
clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
cl1 = clahe.apply(imgg)

cv2.imshow('img',imgg)
cv2.imshow('cl1',cl1)
cv2.waitKey(0)
cv2.destroyAllWindows()


In [ ]:
import numpy as np
import cv2
 
def ComputeHist(img):
    h,w = img.shape
    hist, bin_edge = np.histogram(img.reshape(1,w*h), bins=list(range(257)))
    return hist
    
def ComputeMinLevel(hist, rate, pnum):
    sum = 0
    for i in range(256):
        sum += hist[i]
        if (sum >= (pnum * rate * 0.01)):
            return i
            
def ComputeMaxLevel(hist, rate, pnum):
    sum = 0
    for i in range(256):
        sum += hist[255-i]
        if (sum >= (pnum * rate * 0.01)):
            return 255-i
            
def LinearMap(minlevel, maxlevel):
    if (minlevel >= maxlevel):
        return []
    else:
        newmap = np.zeros(256)
        for i in range(256):    #获取阈值外的像素值 i< minlevel，i> maxlevel
            if (i < minlevel):
                newmap[i] = 0
            elif (i > maxlevel):
                newmap[i] = 255
            else:
                newmap[i] = (i-minlevel)/(maxlevel-minlevel) * 255
        return newmap
        
def CreateNewImg(img):
    h,w,d = img.shape
    newimg = np.zeros([h,w,d])
    for i in range(d):
        imgmin = np.min(img[:,:,i])
        imgmax = np.max(img[:,:,i])
        imghist = ComputeHist(img[:,:,i])
        minlevel = ComputeMinLevel(imghist, 8.3, h*w)
        maxlevel = ComputeMaxLevel(imghist, 2.2, h*w)
        newmap = LinearMap(minlevel,maxlevel)
        if (newmap.size ==0 ):
            continue
        for j in range(h):
            newimg[j,:,i] = newmap[img[j,:, i]]
    return newimg


if __name__ == '__main__':
    img1 = cv2.imread(r"F:\PYcode\Coding\ic\license\source_img\6.jpg")
    img = cv2.resize(img1,(563,750))    
    newimg = CreateNewImg(img)
    cv2.imshow('original_img', img)
    cv2.imshow('new_img', newimg/255)
    cv2.waitKey(0)
    cv2.destroyAllWindows()


In [ ]:
imga  = cv2.imread(r'F:\PYcode\faceinfo\tree.jpg')
B,G,R = cv2.split(imga)

In [ ]:
mb = B.mean()
mg = G.mean()
mr = R.mean()
kb = (mb + mg + mr) / (3 * mb)
kg = (mb + mg + mr) / (3 * mg)
kr = (mb + mg + mr) / (3 * mr)
B = B * kb
G = G * kg
R = R * kr

In [ ]:
new_img = cv2.merge([R,G,B])

In [ ]:
import cv2
import numpy as np
from matplotlib import pyplot as plt

img = cv2.imread(r"F:\PYcode\Coding\ic\license\source_img\6.jpg",0)
# img = cv2.resize(img,(563,750))

img = cv2.medianBlur(img,5)

ret,th1 = cv2.threshold(img,127,255,cv2.THRESH_BINARY)
th2 = cv2.adaptiveThreshold(img,255,cv2.ADAPTIVE_THRESH_MEAN_C,\
            cv2.THRESH_BINARY,11,2)
th3 = cv2.adaptiveThreshold(img,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C,\
            cv2.THRESH_BINARY,11,2)

titles = ['Original Image', 'Global Thresholding (v = 127)',
            'Adaptive Mean Thresholding', 'Adaptive Gaussian Thresholding']
images = [img, th1, th2, th3]

for i in range(4):
    plt.subplot(2,2,i+1),plt.imshow(images[i],'gray')
    plt.title(titles[i])
    plt.xticks([]),plt.yticks([])
plt.show()


In [ ]:
cv2.imwrite("th3.jpg",th3)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
# 颜色各通道的直方图
def calcAndDrawHist(image, color):  
    hist= cv2.calcHist([image], [0], None, [256], [0.0,255.0])  
    minVal, maxVal, minLoc, maxLoc = cv2.minMaxLoc(hist)  
    histImg = np.zeros([256,256,3], np.uint8)  
    hpt = int(0.9* 256);  

    for h in range(256):  
        intensity = int(hist[h]*hpt/maxVal)  
        cv2.line(histImg,(h,256), (h,256-intensity), color)  

    return histImg
import numpy as np
b, g, r = cv2.split(imga)  
histImgB = calcAndDrawHist(b, [255, 0, 0])  
histImgG = calcAndDrawHist(g, [0, 255, 0])  
histImgR = calcAndDrawHist(r, [0, 0, 255])  

cv2.imshow("histImgB", histImgB)  
cv2.imshow("histImgG", histImgG)  
cv2.imshow("histImgR", histImgR)  
cv2.imshow("Img", img)  
cv2.waitKey(0)  
cv2.destroyAllWindows() 

In [21]:
import cv2  
  
img = cv2.imread(r'F:\PYcode\faceinfo\4.jpg')  
gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)  
ret, binary = cv2.threshold(gray,127,255,cv2.THRESH_BINARY)  
  
_, contours, hierarchy = cv2.findContours(binary,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)  
cv2.drawContours(img,contours,-1,(0,0,255),1)  
  
cv2.imshow("img", img)  
cv2.waitKey(0)  
cv2.destroyAllWindows() 

In [25]:

for contour in contours:
    #计算一个简单的边界框
    x,y,w,h = cv2.boundingRect(contour)
    #画出边界框
    cv2.rectangle(img, (x,y), (x+w, y+h),(0,0,255),2)
    #计算包围目标的最小矩形区域
    rect = cv2.minAreaRect(contour)
    box = cv2.boxPoints(rect)
    box = np.int0(box)
    cv2.drawContours(img, [box], 0, (255, 255, 255), 2)
    #cv2.minEnclosingCircle函数返回一个二元组，第一个元组为圆心坐标，第二个为半径
    (x0,y0), radius = cv2.minEnclosingCircle(contour)
    
    center = (int(x0), int(y0))
    radius = int(radius)
    img = cv2.circle(img, center, radius, (0,255,0))

cv2.drawContours(img, contours, -1, (255,0,0), 1)
cv2.imwrite("findcontour.jpg",img)
cv2.imshow("img", img)  
cv2.waitKey(0)  
cv2.destroyAllWindows() 